In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# List of states with URLs, number of pages, and output file names
states_info = [
    {
        'state_name': 'South Bengal',
        'url': "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
        'pages': 5,
        'output_file': 'sb_bus_details.csv'
    },
    {
        'state_name': 'Rajasthan',
        'url': "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
        'pages': 3,
        'output_file': 'rajasthan_bus_details.csv'
    },
    {
        'state_name': 'Telangana',
        'url': "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
        'pages': 3,
        'output_file': 'Telangana_bus_details.csv'
    },
    {
        'state_name': 'UP',
        'url': "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
        'pages': 5,
        'output_file': 'up_bus_details.csv'
    },
    {
        'state_name': 'WB',
        'url': "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometil",
        'pages': 5,
        'output_file': 'wb_bus_details.csv'
    },
    {
        'state_name': 'WB2',
        'url': "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile",
        'pages': 1,
        'output_file': 'wb2_bus_details.csv'
    },
    {
        'state_name': 'Kadamba',
        'url': "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
        'pages': 4,
        'output_file': 'kadamba_bus_details.csv'
    },
    {
        'state_name': 'KAAC',
        'url': "https://www.redbus.in/online-booking/kaac-transport/?utm_source=rtchometile",
        'pages': 2,
        'output_file': 'kaac_bus_details.csv'
    },
    {
        'state_name': 'Kerala',
        'url': "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
        'pages': 2,
        'output_file': 'kerala_bus_details.csv'
    },
    {
        'state_name': 'Himachal',
        'url': "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
        'pages': 5,
        'output_file': 'himachal_bus_details.csv'
    },
    {
        'state_name': 'Assam',
        'url': "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
        'pages': 5,
        'output_file': 'assam_bus_details.csv'
    },
]

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Function to scrape all pages for a given state
def scrape_state_data(state_info):
    all_bus_details = []
    state_name = state_info['state_name']
    url = state_info['url']
    pages = state_info['pages']
    output_file = state_info['output_file']
    
    for page in range(1, pages + 1):
        try:
            driver = initialize_driver()
            load_page(driver, url)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
        
        except Exception as e:
            print(f"Error occurred while accessing page {page} for {state_name}: {str(e)}")
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_bus_details)
    
    # Save the DataFrame to a CSV file
    df.to_csv(output_file, index=False)
    
    # Close the driver
    driver.quit()

# Loop through each state and scrape data
for state_info in states_info:
    scrape_state_data(state_info)



Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/digha-to-durgapur: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7FA9D6CF5+28821]
	(No symbol) [0x00007FF7FA943880]
	(No symbol) [0x00007FF7FA7E578A]
	(No symbol) [0x00007FF7FA8391BE]
	(No symbol) [0x00007FF7FA8394AC]
	(No symbol) [0x00007FF7FA882647]
	(No symbol) [0x00007FF7FA85F33F]
	(No symbol) [0x00007FF7FA87F412]
	(No symbol) [0x00007FF7FA85F0A3]
	(No symbol) [0x00007FF7FA82A778]
	(No symbol) [0x00007FF7FA82B8E1]
	GetHandleVerifier [0x00007FF7FAD0FCED+3408013]
	GetHandleVerifier [0x00007FF7FAD2745F+3504127]
	GetHandleVerifier [0x00007FF7FAD1B63D+3455453]
	GetHandleVerifier [0x00007FF7FAA9BDFB+835995]
	(No symbol) [0x00007FF7FA94EB9F]
	(No symbol) [0x00007FF7FA94A854]
	(No symbol) [0x00007FF7FA94A9ED]
	(No symbol) [0x00007FF7FA93A1D9]
	BaseThreadInitThunk [0x00007FFA3525259D+29]
	RtlUserThreadStart [0x00007FFA3630AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tick